In [3]:
for i in range(8):
    print(i)

0
1
2
3
4
5
6
7


# Data Wrangling Challenge
### Pull and manipulate the API data

The point of this exercise is to try data enrichment with data from external APIs. We are going to take data about car crashes in Monroe County, Indiana from 2003 to 2015 and try to figure out the weather during the accident and how many bars there are in the area. We will work with two different APIs during this challenge:

- Foursquare API
- Visual Crossing API

We will try to find correlations between the severity of crash and weather/number of bars in the area. To indicate the severity of a crash, we will use column `Injury Type`.

## Data

The data for this exercise can be found [here](https://drive.google.com/file/d/1_KF9oIJV8cB8i3ngA4JPOLWIE_ETE6CJ/view?usp=sharing).

Just run the cells below to get your data ready. Little help from us.


In [1]:
import pandas as pd
import os

In [2]:
data = pd.read_csv("C:/Users/Tim/Desktop/lighthouse/w2/d3/Other_data_types_exercise/w2d3challenge.csv", encoding="unicode_escape")
# ========================
# preparing data
data.dropna(subset=['Latitude', 'Longitude'], inplace=True)
# creation of variable with lon and lat together
data['lld'] = data['Latitude'].astype(str) + ',' + data['Longitude'].astype(str)
data = data[data['lld'] != '0.0,0.0']
print(data.shape)
data.head()

(49005, 13)


,Master Record Number,Year,Month,Day,Weekend?,Hour,Collision Type,Injury Type,Primary Factor,Reported_Location,Latitude,Longitude,lld
0,902363382,2015,1,5,Weekday,0.0,2-Car,No injury/unknown,OTHER (DRIVER) - EXPLAIN IN NARRATIVE,1ST & FESS,39.159207,-86.525874,"39.15920668,-86.52587356"
1,902364268,2015,1,6,Weekday,1500.0,2-Car,No injury/unknown,FOLLOWING TOO CLOSELY,2ND & COLLEGE,39.161440,-86.534848,"39.16144,-86.534848"
2,902364412,2015,1,6,Weekend,2300.0,2-Car,Non-incapacitating,DISREGARD SIGNAL/REG SIGN,BASSWOOD & BLOOMFIELD,39.149780,-86.568890,"39.14978027,-86.56889006"
3,902364551,2015,1,7,Weekend,900.0,2-Car,Non-incapacitating,FAILURE TO YIELD RIGHT OF WAY,GATES & JACOBS,39.165655,-86.575956,"39.165655,-86.57595635"
4,902364615,2015,1,7,Weekend,1100.0,2-Car,No injury/unknown,FAILURE TO YIELD RIGHT OF WAY,W 3RD,39.164848,-86.579625,"39.164848,-86.57962482"


# Foursquare API

Foursquare API documentation is [here](https://developer.foursquare.com/)

1. Start a foursquare application and get your keys.
2. For each crash, create the function **get_venues** that will pull bars in the radius of 5km around the crash

#### example
`get_venues('48.146394, 17.107969')`

3. Find a relationship (if there is any) between number of bars in the area and severity of the crash.

HINTs: 
- check out python package "foursquare" (no need to send HTTP requests directly with library `requests`)
- **categoryId** for bars and nightlife needs to be found in the [foursquare API documentation](https://developer.foursquare.com/docs/api-reference/venues/search/)

In [3]:
#set the keys
#foursquare_id = os.environ[""]
#foursquare_secret = os.environ[""]
fs_id = "LI2VHLBCPFJEO1D1O5EMPDXA233B5QP4FGH442ERLK4INCNQ"
fs_s = "BAFOMI5B3PTIJFL5FVJIB0I4ZBQGXYXFPCAUQWHWT5EI3GWA"

In [4]:
#client = foursquare.Foursquare(client_id='fs_id', client_secret='fs_s')

In [15]:
#https://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude/43211266#43211266
#import geopy.distance
import requests
from IPython.display import JSON
import json
from pandas import json_normalize  


#for all crashes

# resdict = {}
# for coord in data["lld"]:
#         param = {
#             client_id = fs_id,
#             client_secret = fs_s,
#             ll = coord,
#             categoryId = "4d4b7105d754a06376d81259",
#             radius = "5000",
#         }
#         resdict[coord] = requests.get(url+param)
# 


def get_venues(x):
    url = "https://api.foursquare.com/v2/venues/search?"
    categoryId = "4bf58dd8d48988d116941735"
    radius = "5000"
    v = "20201231"
    param = f'll={x}&categoryId={categoryId}&radius={radius}&v={v}&client_id={fs_id}&client_secret={fs_s}'
    res = requests.get(url+param)
    result = res.json()
#   df = json_normalize(data=result['response'],record_path='venues')
#   n_bars = df['id'].count()
#   return n_bars
    bars = [venue['name'] for venue in result['response']['venues']]
    num_bars = len(bars)
    return num_bars

In [16]:
get_venues('48.146394, 17.107969')


{'meta': {'code': 429,
  'errorType': 'quota_exceeded',
  'errorDetail': 'Quota exceeded',
  'requestId': '5fffe30fc754223a4e07345b'},
 'response': {}}

In [99]:
#y = result['response']['venues']
#y
df['id'].count()

30

In [112]:
json_normalize(data=result['response'],record_path='venues')['id'].count()

30

In [107]:
#Find a relationship (if there is any) between number of bars in the area and severity of the crash
# def injrel(x = data["lld"]):
#     url = "https://api.foursquare.com/v2/venues/search?"
#     categoryId = "4bf58dd8d48988d116941735"
#     radius = "5000"
#     v = "20201231"
#     param = f'll={x}&categoryId={categoryId}&radius={radius}&v={v}&client_id={fs_id}&client_secret={fs_s}'
#     resdict = {}
#     for coord in x:
#         res = requests.get(url+param)
#         result = res.json()
#         df = json_normalize(data=result['response'],record_path='venues')
#         resdict[coord] = df['id'].count()
#     data['n_bars']=data['lld'].map(resdict)
#     return data.groupby(['Injury Type'])['n_bars'].sum()
# injrel()

In [114]:
n_bars = []
for i in data["lld"]:
    get_venues(i)
    n_bars.append(num_bars)
    data['n_bars'] = n_bars

df = pd.DataFrame([data['Injury Type'],data['n_bars'],data['lld']])
df


# for i in data["lld"]:
#     get_venues(i)
#     data['n_bars'] = json_normalize(data=result['response'],record_path='venues')['id'].count()

# Visual Crossing API

Virtual Crossing API documentation is [here](https://www.visualcrossing.com/resources/documentation/)

1. Sign up for FREE api key if you haven't done that before.
2. For each crush, get the weather for the location and date.
3. Find a relationship between the weather and severity of the crash.

Hints:

* randomly sample only 250 or so (due to API limits), or pull weather only for smaller sample of crashes
* for sending HTTP requests check out "requests" library [here](http://docs.python-requests.org/en/master/)


In [193]:
import requests
import time
api_key = "NJPND9WTJDMA8PEWHCG65VFNJ"
x = data['lld']
y = df['Datetime']=(pd.to_datetime(df['Year'].astype(str) + '-' +
                                  df['Month'].astype(str) + '-' +
                                  df['Date'].astype(str))

               

In [ ]:
#https://www.visualcrossing.com/weather/weather-data-services#/editDataDefinition
def get_weather(x,y): #location={lat}%2C{lon}
    url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/location={lld}/date1={Datetime}?unitGroup=metric&key=NJPND9WTJDMA8PEWHCG65VFNJ&include=obs"
    res = requests.get(url)
    result = res.json()
    weathdict = {}
    #for loc in x:
        #for time in y:
    precip = [weather['precip'] for weather in result['days']]
    #could pick these too
    #snow = [weather2['snowdepth'] for weather2 in result['days']]
    #vis = [weather3['visibility'] for weather3 in result['days']]
    return precip

In [ ]:
weath = []
for j,k in data[["lld","Datetime"]]:
    get_weather(j,k)
    weath.append(precip)
    data['weath'] = weath
    
df2 = pd.DataFrame([data['Injury Type'],data['weath'],data['lld']])
df2